# Surface restructuring event analysis of LAMMPS MD trajectory:
## Preparation of LAMMPS NEB calculation

## Header

In [ ]:
import lmpdump as lmpdump    # Use lmpdump.py parser
import os
import math
import numpy as np
import time
from tqdm import tqdm
from sitator.util import PBCCalculator

pwd = os.getcwd()

print('Please cite DOI: 10.1021/acs.jpcc.9b04863.')
print('This script detects interlayer surface restructuring events from LAMMPS MD trajectory and generates images for subsequent LAMMPS NEB calculation of each event.')
print('This script is interactive and requires user input. Please read carefully before proceeding:\n')

print('Note 1: Intended only for interlayer surface restructuring events in FCC metals.')
print('Note 2: Intended only for periodic slab models with vacuum along the z-direction.')
print('Note 3: Intended only for LAMMPS NVT simulations.')
print('Note 4: Intended only for estimation of the event statistics, up to bimetallic systems. Please confirm with visual inspection as needed.\n')
print('Note 5: Must have used DFT unit cell if planning on subsequent DFT optimization (via NEB2DFT.ipynb).')

print('Note 5: Requires a LAMMPS DATA file containing the equilibrated box dimensions.')
print('Note 6: Requires two trajectories, ordered by atom ID & unscaled: (1) Clamped (via lmpclamp.py), dumped every ps, xy-unwrapped; (2) Raw, dumped every 0.1 ps, wrapped.')
print('Note 7: Requires lmpdump.py to load the LAMMPS trajectories.\n')
print('Note 8: Requires PBC calculator from sitator package.\n')

## Load LAMMPS trajectories

In [ ]:
# LAMMPS custom-style dump file - Clamped trajectory
name = input('Enter the name of the clamped LAMMPS trajectory file (ID, type, xu, yu, z, c_cn) dumped every ps : \nWarning: Must be clamped (via lmpclamp.py), ordered by atom ID, unscaled & xy-unwrapped! Please quit now if not so.\n')
print('Loading trajectory-1...')
file = pwd + '/' + name
xyz1 = lmpdump.lmpdump(file, loadmode='all')
print('\n')

print('Loading time steps...')
step = []
for s in xyz1.finaldict.keys():    # keys = time steps
    step.append(s)
print('Done.\n')

size = np.array(step).size    # Number of time steps
N_dump = xyz1.finaldict[0][1].shape[0]    # Number of atoms dumped
xlo_ext = xyz1.finaldict[0][0][0]    # x_ext = x + xy
xhi_ext = xyz1.finaldict[0][0][1]

# LAMMPS atom-style dump file - Raw trajectory
name = input('\n Enter the name of the raw LAMMPS trajectory file (ID, type, x, y, z) dumped every 0.1 ps : \nWarning: Must be raw, ordered by atom ID, unscaled & wrapped! Please quit now if not so.\n')
print('Loading trajectory-2...')
file = pwd + '/' + name
xyz2 = lmpdump.lmpdump(file, loadmode='all')
print('\n')

## Extract unit cell information

In [ ]:
# Load LAMMPS data file after NPT equilibration
name = input('Enter the name of the LAMMPS DATA file containing the equilibrated box dimensions : ')
file_eq = pwd + '/' + name
log = open(file_eq,'r')
log_lines = log.readlines()
log_lines = [line.split() for line in log_lines]

for line_index, line in enumerate(log_lines):
    if line:
        for l in line:
            
            if l == 'atoms':
                N_atom = int(line[0])
                
            elif l == 'types':
                N_type = int(line[0])

            elif l == 'xlo':
                xlo = float(line[0])
                xhi = float(line[1])
            
            elif l == 'ylo':
                ylo = float(line[0])
                yhi = float(line[1])
            
            elif l == 'zlo':
                zlo = float(line[0])
                zhi = float(line[1])
            
            elif l == 'xy':
                xy = float(line[0])
                xz = float(line[1])
                yz = float(line[2])
            
            elif l == 'Atoms':
                head = line_index + 1
                start = line_index + 2
            
            elif l == 'Velocities':
                end = line_index - 1

Lx = xhi - xlo    # Box dimensions
Ly = yhi - ylo
Lz = zhi - zlo
tan = Ly/xy

xyz = []    # N*3 matrix
for line in log_lines[start:end]:

    line[:2] = np.array(line[:2]).astype(int)    # Atom ID, atom type
    line[2:5] = np.array(line[2:5]).astype(float)    # x, y, z
    line[5:8] = np.array(line[5:8]).astype(int)    # ix, iy, iz (inverse sign)

    x = line[2] + line[5]*Lx + line[6]*xy    # Wrapped coordinates
    y = line[3] + line[6]*Ly
    z = line[4] + line[7]*Lz
    
    xyz.append([x,y,z])
xyz = np.array(xyz)

ID2 = int(input('Enter the ID of a nearest-neighbor atom that lies in the same layer as atom #1 : '))
ID3 = int(input('Enter the ID of another nearest-neighbor atom that lies in the same layer as atom #1 : '))
ID4 = int(input('Enter the ID of an atom that lies one layer above atom #1 : '))

for atom, coord in enumerate(xyz):
    
    if atom+1 == 1:
        atom1 = coord    # atom1 = Reference atom
    
    elif atom+1 == ID2:
        atom2 = coord    # atom2 = 1st coplanar atom

    elif atom+1 == ID3:
        atom3 = coord    # atom3 = 2nd coplanar atom
        
    elif atom+1 == ID4:
        atom4 = coord    # atom4 = Atom one layer above

r12 = np.linalg.norm(np.subtract(atom1,atom2))
r13 = np.linalg.norm(np.subtract(atom1,atom3))
dr_avg = np.mean([r12, r13])    # In-plane NN distance

a1 = np.subtract(atom2,atom1)    # 1st intralayer vector
a1 = a1 / np.linalg.norm(a1)

a2 = np.subtract(atom3,atom1)    # 2nd intralayer vector
a2 = a2 / np.linalg.norm(a2)

a3 = np.cross(a1,a2)    # Normal vector
if a3[2] < 0:
    a3 = -a3    # Ensure normal vector along +z
a3 = a3 / np.linalg.norm(a3)

dn_avg = np.dot(atom4,a3) - np.dot(atom1,a3)    # Interlayer distance; n = normal component

N_slab = int(input('Enter the number of layers in the slab, excluding the deposit/adsorbate layer(s) : '))
if xyz1.finaldict[0][1]['id'][0] != 1:    # Is bottommost layer muted?
    N_slab -= 1

z_layer = []    # Clamped layer heights
for atom in range(1,N_dump):
    z = xyz1.finaldict[step[-1]][1]['z'][atom]    # Use last frame
    if z not in z_layer:
        z_layer.append(z)
N_layer = len(z_layer)

log.close()

print('Unit cell information & normal direction extracted.\n')

## Event detection

In [ ]:
print('Parsing trajectory for restructuring events...')
t_begin = time.perf_counter()

file_ev = pwd + '/event_flag.txt'    # List of time steps flagged as event points
out_ev = open(file_ev,'w')

header1 = ' tf = final time (flagged step) \n dt = event duration \n ni = initial normal component \n nf = final normal \n CNi = initial coordination number \n CNf = final coordination number \n \n'
header2 = 'tf (ps) \t dt (ps) \t ID \t type \t ni \t nf \t CNi \t CNf \n'
out_ev.write(header1)
out_ev.write(header2)

skip_index = [0, 1, 2, size-2, size-1]    # Skip time steps at the edges of the trajectory

for s_index, s in enumerate(tqdm(step)):
    
    if s_index in skip_index:
        continue

    ev_atom = []    # List of active atoms detected
    for atom in range(0,N_dump):
            
        # f0 = current time step (final 0)
        # f1 = 1 step after (final 1)
        # f2 = 2 steps after (final 2)
        # i1 = 1 step before (initial 1)
        # i2 = 2 steps before (initial 2)
        # i3 = 3 steps before (initial 3)
        
        ID = xyz1.finaldict[s][1]['id'][atom]
        Type = xyz1.finaldict[s][1]['type'][atom]
        
        CNf0 = xyz1.finaldict[s][1]['c_cn'][atom]
        CNi1 = xyz1.finaldict[step[s_index-1]][1]['c_cn'][atom]
        CNi2 = xyz1.finaldict[step[s_index-2]][1]['c_cn'][atom]
        
        xf0 = xyz1.finaldict[s][1]['xu'][atom]
        yf0 = xyz1.finaldict[s][1]['yu'][atom]
        zf0 = xyz1.finaldict[s][1]['z'][atom]
        rf0 = np.array([xf0, yf0, zf0])
        nf0 = np.dot(rf0,a3)
        xf0_prj = xf0 - yf0/tan    # Tilted projection of x
        Nxf0 = math.floor(xf0_prj/Lx)    # x image number
        Nyf0 = math.floor(yf0/Ly)    # y image number
    
        xf1 = xyz1.finaldict[step[s_index+1]][1]['xu'][atom]
        yf1 = xyz1.finaldict[step[s_index+1]][1]['yu'][atom]
        zf1 = xyz1.finaldict[step[s_index+1]][1]['z'][atom]
        rf1 = np.array([xf1, yf1, zf1])
        nf1 = np.dot(rf1,a3)
        xf1_prj = xf1 - yf1/tan
        Nxf1 = math.floor(xf1_prj/Lx)
        Nyf1 = math.floor(yf1/Ly)
        
        xf2 = xyz1.finaldict[step[s_index+2]][1]['xu'][atom]
        yf2 = xyz1.finaldict[step[s_index+2]][1]['yu'][atom]
        zf2 = xyz1.finaldict[step[s_index+2]][1]['z'][atom]
        rf2 = np.array([xf2, yf2, zf2])
        nf2 = np.dot(rf2,a3)
        xf2_prj = xf2 - yf2/tan
        Nxf2 = math.floor(xf2_prj/Lx)
        Nyf2 = math.floor(yf2/Ly)
        
        xi1 = xyz1.finaldict[step[s_index-1]][1]['xu'][atom]
        yi1 = xyz1.finaldict[step[s_index-1]][1]['yu'][atom]
        zi1 = xyz1.finaldict[step[s_index-1]][1]['z'][atom]
        ri1 = np.array([xi1, yi1, zi1])
        ni1 = np.dot(ri1,a3)
        xi1_prj = xi1 - yi1/tan
        Nxi1 = math.floor(xi1_prj/Lx)
        Nyi1 = math.floor(yi1/Ly)
        
        xi2 = xyz1.finaldict[step[s_index-2]][1]['xu'][atom]
        yi2 = xyz1.finaldict[step[s_index-2]][1]['yu'][atom]
        zi2 = xyz1.finaldict[step[s_index-2]][1]['z'][atom]
        ri2 = np.array([xi2, yi2, zi2])
        ni2 = np.dot(ri2,a3)
        xi2_prj = xi2 - yi2/tan
        Nxi2 = math.floor(xi2_prj/Lx)
        Nyi2 = math.floor(yi2/Ly)
        
        xi3 = xyz1.finaldict[step[s_index-3]][1]['xu'][atom]
        yi3 = xyz1.finaldict[step[s_index-3]][1]['yu'][atom]
        zi3 = xyz1.finaldict[step[s_index-3]][1]['z'][atom]
        ri3 = np.array([xi3, yi3, zi3])
        ni3 = np.dot(ri3,a3)
        xi3_prj = xi3 - yi3/tan
        Nxi3 = math.floor(xi3_prj/Lx)
        Nyi3 = math.floor(yi3/Ly)
    
        dni1 = nf0 - ni1    # Change in normal from the past
        dni2 = nf0 - ni2
        dni3 = nf0 - ni3
        
        dnf1 = nf1 - nf0    # Change in normal in the future
        dnf2 = nf2 - nf0
        
        # Test 1 step in the past
        # Criterion: Change by more than 90% of interlayer distance
        # History check: Must not revert back 1 or 2 steps in the past & in the future
        
        if (abs(dni1) > 0.90*dn_avg) and (abs(dni2) > 0.90*dn_avg) and (abs(dnf1) < 0.10*dn_avg) and (abs(dnf2) < 0.10*dn_avg):
            
            ev_atom.append(atom)
            dt = 1
            
            xf0 -= xy * Nyf0    # Wrap y first
            yf0 -= Ly * Nyf0
            
            xi1 -= xy * Nyi1
            yi1 -= Ly * Nyi1
            
            if (Nxf0 != 0) or (Nxi1 != 0):
            
                xf0 -= Lx * np.maximum(Nxf0,Nxi1)    # Shift initial & final x together toward the unit cell
                rf0 = np.array([xf0, yf0, zf0])
                nf0 = np.dot(rf0,a3)
            
                xi1 -= Lx * np.maximum(Nxf0,Nxi1)
                ri1 = np.array([xi1, yi1, zi1])
                ni1 = np.dot(ri1,a3)
            
            line = '%i\t%i\t%i\t%i\t%f\t%f\t%i\t%i\n'%(s_index, dt, ID, Type, ni1, nf0, CNi1, CNf0)
            out_ev.write(line)
            
            for aux in range(0,N_dump):    # Detect any auxiliary atom that may move together
                if aux not in ev_atom:    # Test only if not flagged before
                    
                    # Auxiliary atom labeled with "i" & "f" only
                    
                    ID = xyz1.finaldict[s][1]['id'][aux]
                    Type = xyz1.finaldict[s][1]['type'][aux]
                    
                    CNf = xyz1.finaldict[s][1]['c_cn'][aux]
                    CNi = xyz1.finaldict[step[s_index-1]][1]['c_cn'][aux]
    
                    xf = xyz1.finaldict[s][1]['xu'][aux]
                    yf = xyz1.finaldict[s][1]['yu'][aux]
                    zf = xyz1.finaldict[s][1]['z'][aux]
                    rf = np.array([xf, yf, zf])
                    nf = np.dot(rf,a3)
                    xf_prj = xf - yf/tan
                    Nxf = math.floor(xf_prj/Lx)
                    Nyf = math.floor(yf/Ly)
                    
                    xi = xyz1.finaldict[step[s_index-1]][1]['xu'][aux]
                    yi = xyz1.finaldict[step[s_index-1]][1]['yu'][aux]
                    zi = xyz1.finaldict[step[s_index-1]][1]['z'][aux]
                    ri = np.array([xi, yi, zi])
                    ni = np.dot(ri,a3)
                    xi_prj = xi - yi/tan
                    Nxi = math.floor(xi_prj/Lx)
                    Nyi = math.floor(yi/Ly)
    
                    dx = xf - xi
                    dy = yf - yi
                    dz = zf - zi                        
                    dr = math.sqrt(dx**2 + dy**2 + dz**2)    # Change in position, rather than normal
                    
                    if dr > 0.80*dr_avg:    # Criterion: Change by more than 80% of the NN distance
                        ev_atom.append(aux)
                        
                        # Auxiliary atom must be wrapped toward the active atom

                        xf -= xy * Nyf    # Wrap y first
                        yf -= Ly * Nyf
                        
                        xi -= xy * Nyi
                        yi -= Ly * Nyi
                        
                        dxf = (xf - xf0)/Lx    # Extent of x deviation from the active atom
                        dxi = (xi - xi1)/Lx
                        
                        if (abs(dxf) > 0.70) or (abs(dxi) > 0.70):    # Wrap if x deviation is more than 70% of the unit cell x dimension
                                
                            xf -= Lx * round(dxf)
                            rf = np.array([xf, yf, zf])
                            nf = np.dot(rf,a3)
                            
                            xi -= Lx * round(dxi)
                            ri = np.array([xi, yi, zi])
                            ni = np.dot(ri,a3)

                        line = '%i\t%i\t%i\t%i\t%f\t%f\t%i\t%i\n'%(s_index, dt, ID, Type, ni, nf, CNi, CNf)
                        out_ev.write(line)
        
        # If no change 1 step in the past, test 2 steps in the past (change may be gradual)
        elif (abs(dni2) > 0.90*dn_avg) and (abs(dni3) > 0.90*dn_avg) and (abs(dnf1) < 0.10*dn_avg) and (abs(dnf2) < 0.10*dn_avg): 
            
            ev_atom.append(atom)
            dt = 2
            
            xf0 -= xy * Nyf0
            yf0 -= Ly * Nyf0
            
            xi2 -= xy * Nyi2
            yi2 -= Ly * Nyi2
    
            if (Nxf0 != 0) or (Nxi2 != 0):
            
                xf0 -= Lx * np.maximum(Nxf0,Nxi2)
                rf0 = np.array([xf0, yf0, zf0])
                nf0 = np.dot(rf0,a3)
            
                xi2 -= Lx * np.maximum(Nxf0,Nxi2)
                ri2 = np.array([xi2, yi2, zi2])
                ni2 = np.dot(ri2,a3)
            
            line = '%i\t%i\t%i\t%i\t%f\t%f\t%i\t%i\n'%(s_index, dt, ID, Type, ni2, nf0, CNi2, CNf0)
            out_ev.write(line)
    
            for aux in range(0,N_dump):
                if aux not in ev_atom:
                    
                    ID = xyz1.finaldict[s][1]['id'][aux]
                    Type = xyz1.finaldict[s][1]['type'][aux]
                    
                    CNf = xyz1.finaldict[s][1]['c_cn'][aux]
                    CNi = xyz1.finaldict[step[s_index-1]][1]['c_cn'][aux]
    
                    xf = xyz1.finaldict[s][1]['xu'][aux]                            
                    yf = xyz1.finaldict[s][1]['yu'][aux]
                    zf = xyz1.finaldict[s][1]['z'][aux]
                    rf = np.array([xf, yf, zf])
                    nf = np.dot(rf,a3)
                    xf_prj = xf - yf/tan
                    Nxf = math.floor(xf_prj/Lx)
                    Nyf = math.floor(yf/Ly)
      
                    xi = xyz1.finaldict[step[s_index-2]][1]['xu'][aux]
                    yi = xyz1.finaldict[step[s_index-2]][1]['yu'][aux]
                    zi = xyz1.finaldict[step[s_index-2]][1]['z'][aux]
                    ri = np.array([xi, yi, zi])
                    ni = np.dot(ri,a3)
                    xi_prj = xi - yi/tan
                    Nxi = math.floor(xi_prj/Lx)
                    Nyi = math.floor(yi/Ly)
    
                    dx = xf - xi
                    dy = yf - yi
                    dz = zf - zi                        
                    dr = math.sqrt(dx**2 + dy**2 + dz**2)
                    
                    if dr > 0.80*dr_avg:
                        ev_atom.append(aux)
                        
                        xf -= xy * Nyf
                        yf -= Ly * Nyf
                        
                        xi -= xy * Nyi
                        yi -= Ly * Nyi
                        
                        dxf = (xf - xf0)/Lx
                        dxi = (xi - xi2)/Lx
                        
                        if (abs(dxf) > 0.70) or (abs(dxi) > 0.70):

                            xf -= Lx * round(dxf)
                            rf = np.array([xf, yf, zf])
                            nf = np.dot(rf,a3)

                            xi -= Lx * round(dxi)
                            ri = np.array([xi, yi, zi])
                            ni = np.dot(ri,a3)
                            
                        line = '%i\t%i\t%i\t%i\t%f\t%f\t%i\t%i\n'%(s_index, dt, ID, Type, ni, nf, CNi, CNf)
                        out_ev.write(line)
                            
out_ev.close()

t_end = time.perf_counter()
print('Event detection done in %.2f' % (t_end - t_begin) + 's > event_flag.txt\n')

## Event clustering

In [ ]:
ev = open(file_ev,'r')    # Load list of time steps flagged as event points
ev_lines = ev.readlines()
ev_lines = [line.split() for line in ev_lines]

for line_index, line in enumerate(ev_lines):
    if line_index > 7:
        line[0:4] = np.array(line[0:4]).astype(int)    # Time step, duration, atom ID, atom type
        line[4:6] = np.array(line[4:6]).astype(float)    # Initial normal, final normal
        line[6:8] = np.array(line[6:8]).astype(int)    # Initial coordination, final coordination

ev.close()

file_cluster = pwd + '/event_cluster.txt'    # List of events
out_cluster = open(file_cluster,'w')

# List active atoms first, followed by event #, final time, and duration
header1 = ' tf = final time (flagged step) \n dt = event duration \n ni = initial normal component \n nf = final normal component \n CNi = initial coordination number \n CNf = final coordination number \n \n'
header2 = '\t ID \t type \t ni \t nf \t CNi \t CNf \n event # \t tf (ps) \t dt (ps) \n \n'
out_cluster.write(header1)
out_cluster.write(header2)

ev_no = 0
ev_atom = []    # List of active atoms for each event
ev_flag = []    # List of flags for each event

for line_index, line in enumerate(ev_lines):

    if line_index < 8:
        continue
    
    elif line_index == 8:
        
        ev_flag.append(line)
        ti = line[0] - line[1]    # ti = initial time
        ID = line[2]
        ev_atom.append(ID)
    
    else:
        
        # If a step is within 3 steps of the previously flagged step, it is part of the same event
        if line[0] - ev_lines[line_index-1][0] < 3:
            
            ev_flag.append(line)
            ID = line[2]
            
            if ID not in ev_atom:
                ev_atom.append(ID)

            if line_index == len(ev_lines)-1:    # If at the last entry, finish clustering
                
                ev_no += 1
                tf = ev_lines[line_index-1][0]

                if tf == ti:
                    dt = ev_lines[line_index-1][1]
                else:
                    dt = tf - ti

                for atom in ev_atom:
                    atom_flag = []    # List of flags for each atom
                    
                    for flag in ev_flag:
                        ID = flag[2]
                        
                        if ID == atom:
                            atom_flag.append(flag)
                    
                    for flag_index, flag in enumerate(atom_flag):
                        
                        ID = flag[2]
                        Type = flag[3]
                        ni = flag[4]
                        nf = flag[5]
                        CNi = flag[6]
                        CNf = flag[7]
                        dn = nf - ni
                        
                        if abs(dn) < 0.10*dn_avg:    # If normal does not change, print only if at the last flag
                            
                            if flag_index != len(atom_flag)-1:
                                continue
                            
                            else:
                                out = '\t%i\t%i\t%f\t%f\t%i\t%i\n'%(ID, Type, ni, nf, CNi, CNf)
                                out_cluster.write(out)
                        
                        elif abs(dn) > 0.90*dn_avg:    # If normal changes, print that flag and move on to the next atom
                            out = '\t%i\t%i\t%f\t%f\t%i\t%i\n'%(ID, Type, ni, nf, CNi, CNf)
                            out_cluster.write(out)
                            break

                out = '%i\t%i\t%i\n\n'%(ev_no, tf, dt)    # Print the event number, time step, and duration
                out_cluster.write(out)
                    
        # If a step is after 3 steps or more from the previously flagged step, it is part of a different event, so finish clustering
        else:

            ev_no += 1
            tf = ev_lines[line_index-1][0]

            if tf == ti:
                dt = ev_lines[line_index-1][1]
            else:
                dt = tf - ti

            for atom in ev_atom:
                atom_flag = []
                
                for flag in ev_flag:
                    ID = flag[2]
                    
                    if ID == atom:
                        atom_flag.append(flag)
                
                for flag_index, flag in enumerate(atom_flag):
                    
                    ID = flag[2]
                    Type = flag[3]
                    ni = flag[4]
                    nf = flag[5]
                    CNi = flag[6]
                    CNf = flag[7]
                    dn = nf - ni
                    
                    if abs(dn) < 0.10*dn_avg:
                        
                        if flag_index != len(atom_flag)-1:
                            continue
                        
                        else:
                            out = '\t%i\t%i\t%f\t%f\t%i\t%i\n'%(ID, Type, ni, nf, CNi, CNf)
                            out_cluster.write(out)
                    
                    elif abs(dn) > 0.90*dn_avg:
                        out = '\t%i\t%i\t%f\t%f\t%i\t%i\n'%(ID, Type, ni, nf, CNi, CNf)
                        out_cluster.write(out)
                        break
            
            out = '%i\t%i\t%i\n\n'%(ev_no, tf, dt)
            out_cluster.write(out)
            
            ev_atom = []    # Reset for the next event
            ev_flag = []
            
            ev_flag.append(line)
            ti = line[0] - line[1]
            ID = line[2]
            
            if ID not in ev_atom:
                ev_atom.append(ID)

out_cluster.close()

print('Event clustering done > event_cluster.txt\n')

## Event localization

In [ ]:
pbc = PBCCalculator(lat)    # Periodic boundary condition

cluster = open(file_cluster,'r')    # Load event clusters
cluster_lines = cluster.readlines()
cluster_lines = [line.split() for line in cluster_lines]

file_ls = pwd + '/event_list.txt'    # List of localized events
out_ls = open(file_ls,'w')

out_ls.write(header1)
out_ls.write(header2)

for line_index, line in enumerate(cluster_lines):
    if line and (line_index > 10):
        if len(line) == 6:
            line[0:2] = np.array(line[0:2]).astype(int)    # Atom ID, atom type
            line[2:4] = np.array(line[2:4]).astype(float)    # Initial normal, final normal
            line[4:6] = np.array(line[4:6]).astype(int)    # Initial coordination, final coordination
        else:
            line[0:3] = np.array(line[0:3]).astype(int)    # event number, time step, duration

line_cont = 10    # Line number continuation
N_splice = 0    # Number of events spliced
while line_cont < len(cluster_lines)-2:
    
    ev_atom = []    # List of event atoms
    for line_index, line in enumerate(cluster_lines):
        if line and (line_index > line_cont):
            if len(line) == 6:
                ev_atom.append(int(line[0]))
            else:
                ev_no = line[0]
                tf = line[1]
                dt = line[2]
                break
    
    ev_coord = []    # List of Cartesian coordinates for event atoms
    for ID in ev_atom:
        for atom in range(0,N_dump):
            if xyz1.finaldict[0][1]['id'][atom] == ID:
                index = atom
                x = xyz1.finaldict[step[tf]][1]['xu'][index]
                y = xyz1.finaldict[step[tf]][1]['yu'][index]
                z = xyz1.finaldict[step[tf]][1]['z'][index]
                r = np.array([x, y, z])
                ev_coord.append(r)
                break
    ev_coord = np.array(ev_coord)
        
    ev_rij = pbc.pairwise_distances(ev_coord)    # Pairwise distance matrix for event atoms
    ev_connectivity = (ev_rij <= 2.0*dr_avg)    # Connectivity matrix (T/F) within a distance cutoff
    
    from scipy.sparse.csgraph import connected_components
    N_group, ls_group = connected_components(ev_connectivity)    # N_group = number of groups; ls_group = list of groups for each event atom
    
    if N_group == 1:    # No splicing needed
        
        for line_index, line in enumerate(cluster_lines):
            if line and (line_index > line_cont):
                if len(line) == 6:
                    out = '\t%i\t%i\t%f\t%f\t%i\t%i\n'%(line[0], line[1], line[2], line[3], line[4], line[5])
                    out_ls.write(out)
                else:
                    out = '%i\t%i\t%i\n\n'%(ev_no, tf, dt)
                    out_ls.write(out)
                    line_cont = line_index
                    break
    
    else:    # Splicing needed
        
        N_splice += 1
        for group_index, group in enumerate(range(0,N_group)):
            
            ev_group = []    # List of event atoms for each group
            for atom_index, atom_group in ls_group:
                if atom_group == group:
                    ev_group.append(ev_atom[atom_index])
            
            for line_index, line in enumerate(cluster_lines):    # Check whether group is z-active
                if line and (line_index > line_cont):
                    if len(line) == 6:
                        if line[0] in ev_group:
                            if abs(atom[2] - atom[3]) > 0.50*dn_avg:
                                z_active = True
                                break
                    else:
                        z_active = False
                        break
            
            if z_active:    # Write only if group is z-active
            
                for line_index, line in enumerate(cluster_lines):
                    if line and (line_index > line_cont):
                        if len(line) == 6:
                            if line[0] in ev_group:
                                out = '\t%i\t%i\t%f\t%f\t%i\t%i\n'%(line[0], line[1], line[2], line[3], line[4], line[5])
                                out_ls.write(out)
                        else:
                            ev_no_new = str(ev_no) + '-' + str(group_index+1)
                            out = '%s\t%i\t%i\n\n'%(ev_no_new, tf, dt)
                            out_ls.write(out)
                            if group_index == N_group-1:
                                line_cont = line_index
                            break                

cluster.close()
out_ls.close()

print('Event localization done: %i ' %(N_splice) + 'events spliced for simultaneous events > event_list.txt\n')

## Event classification

In [ ]:
file_ls = pwd + '/event_list.txt'    # List of events
ls = open(file_ls,'r')    # Load list of events
ls_lines = ls.readlines()
ls_lines = [line.split() for line in ls_lines]

file_cls = pwd + '/event_class.txt'    # List of event classes
out_cls = open(file_cls,'w')

disclaimer = 'Intended only for estimation of the event statistics. Please confirm with visual inspection as needed.\n'
header = 'ID \t Type \t Site_i \t Site_f \n\n'
out_cls.write(disclaimer)
out_cls.write(header)

Lv1_1 = 'Isolated adatom'
Lv1_2 = 'Edge/kink/vacancy'
Lv1_3 = 'Deposit intralayer'
Lv0_1 = 'Surface intralayer'
Lv0_2 = 'Subdeposit'
Lv_sub = 'Subsurface'

for line_index, line in enumerate(ls_lines):
    if line and (line_index > 9):
        if len(line) == 6:
            line[0:2] = np.array(line[0:2]).astype(int)    # Atom ID, atom type
            line[2:4] = np.array(line[2:4]).astype(float)    # Initial normal, final normal
            line[4:6] = np.array(line[4:6]).astype(int)    # Initial coordination, final coordination
        else:
            line[0:3] = np.array(line[0:3]).astype(int)   # Event #, time step, duration
    
line_cont = 9    # Line number continuation
while line_cont < len(ls_lines)-2:
    
    ev_atom = []    # List of atom attributes for each event
    for line_index, line in enumerate(ls_lines):
        if line and (line_index > line_cont):
            if len(line) == 6:
                ev_atom.append(line)
            else:
                ev_no = line[0]
                tf = line[1]
                dt = line[2]
                line_cont = line_index
                break
    
    out = '%s%i\n%s%i%s\n%s%i%s\n'%('Event #', ev_no, 'tf = ', tf, ' ps', 'dt = ', dt, ' ps')
    out_cls.write(out)
    
    N = len(ev_atom)    # Number of active atoms
    index_z = []    # Line indices of z-active atoms
    index_aux = []    # Line indices of auxiliary atoms
    
    for atom_index, atom in enumerate(ev_atom):
        if abs(atom[2] - atom[3]) > 0.50*dn_avg:
            index_z.append(atom_index)
        else:
            index_aux.append(atom_index)
    
    N_z = len(index_z)    # Number of z-active atoms
    N_aux = len(index_aux)    # Number of auxiliary atoms
    
    site = []    # Initial & final site & type matrix
    for atom in ev_atom:
        
        ID = atom[0]
        Type = atom[1]
        n = [atom[2], atom[3]]
        CN = [atom[4], atom[5]]
        atom_site = [[],[]]
        
        for i in range(0,2):
            
            if n[i] < z_layer[N_slab-1]:
                atom_site[i] = Lv_sub    # Subsurface
            
            elif n[i] == z_layer[N_slab-1]:
                
                if 11 <= CN[i] <= 12:
                    atom_site[i] = Lv0_2    # Subdeposit
                
                elif 8 <= CN[i] <= 10:
                    atom_site[i] = Lv0_1    # Surface intralayer
            
            elif n[i] > z_layer[N_slab-1]:
                
                if 8 <= CN[i] <= 12:
                    atom_site[i] = Lv1_3    # Deposit intralayer
                
                elif 4 <= CN[i] <= 7:
                    atom_site[i] = Lv1_2    # Edge/kink/vacancy
            
                elif 1 <= CN[i] <= 3:
                    atom_site[i] = Lv1_1    # Isolated adatom
        
        site.append(atom_site)
        out = '%i\t%i\t%s %s %s\n'%(ID, Type, atom_site[0], '==>', atom_site[1])
        out_cls.write(out)

    # More than 5 active atoms = Indeterminate
    if N > 5:
        out = 'Visual inspection needed: More than 5 active atoms. Most likely conjoined events or intralayer shift.\n'
        out_cls.write(out)

    # 1 z-active atom = Popout vs. insertion vs. step motion
    elif N_z == 1:
        
        Type = ev_atom[index_z[0]][1]
        
        if Lv_sub in site[index_z[0]]:
            out = 'Class = Interlayer transfer\nAtom type = ' + str(Type) + '\n'
            out_cls.write(out)
        
        elif ((site[index_z[0]][0] == Lv0_1) or (site[index_z[0]][0] == Lv0_2)) and ((site[index_z[0]][1] == Lv1_1) or (site[index_z[0]][1] == Lv1_2) or (site[index_z[0]][1] == Lv1_3)):
            out = 'Class = Popout\nAtom type = ' + str(Type) + '\n'
            out_cls.write(out)
        
        elif ((site[index_z[0]][0] == Lv1_1) or (site[index_z[0]][0] == Lv1_2) or (site[index_z[0]][0] == Lv1_3)) and ((site[index_z[0]][1] == Lv0_1) or (site[index_z[0]][1] == Lv0_2)):
            out = 'Class = Vacancy insertion\nAtom type = ' + str(Type) + '\n'
            out_cls.write(out)
        
        elif ((site[index_z[0]][0] == Lv1_1) or (site[index_z[0]][0] == Lv1_2) or (site[index_z[0]][0] == Lv1_3)) and ((site[index_z[0]][1] == Lv1_1) or (site[index_z[0]][1] == Lv1_2) or (site[index_z[0]][1] == Lv1_3)):
            
            dn = ev_atom[index_z[0]][3] - ev_atom[index_z[0]][2]
            
            if N_aux != 0:
                if dn > 0:
                    out = 'Class = Exchange ascent\nAtom type = ' + str(Type) + '\n'
                    out_cls.write(out)
                else:
                    out = 'Class = Exchange descent\nAtom type = ' + str(Type) + '\n'
                    out_cls.write(out)
            
            else:
                if dn > 0:
                    out = 'Class = Hopping ascent\nAtom type = ' + str(Type) + '\n'
                    out_cls.write(out)
                else:
                    out = 'Class = Hopping descent\nAtom type = ' + str(Type) + '\n'
                    out_cls.write(out)
    
    # 2 z-active atoms = Popout vs. insertion vs. adatom exchange
    elif N_z == 2:
        
        ID1 = ev_atom[index_z[0]][0]
        ID2 = ev_atom[index_z[1]][0]
        Type1 = ev_atom[index_z[0]][1]
        Type2 = ev_atom[index_z[1]][1]
        dn1 = ev_atom[index_z[0]][3] - ev_atom[index_z[0]][2]
        dn2 = ev_atom[index_z[1]][3] - ev_atom[index_z[1]][2]

        if Lv_sub in site[index_z[0]]:
            out = 'Class = Interlayer transfer\nAtom type = ' + str(Type1) + '\n'
            out_cls.write(out)
            
        elif Lv_sub in site[index_z[1]]:
            out = 'Class = Interlayer transfer\nAtom type = ' + str(Type2) + '\n'
            out_cls.write(out)
        
        elif (dn1 > 0) and (dn2 > 0):
            
            if ((site[index_z[0]][0] == Lv0_1) or (site[index_z[0]][0] == Lv0_2)):
                out = 'Class = Popout\nAtom type = ' + str(Type1) + '\n'
                out_cls.write(out)
                
            elif ((site[index_z[1]][0] == Lv0_1) or (site[index_z[1]][0] == Lv0_2)):
                out = 'Class = Popout\nAtom type = ' + str(Type2) + '\n'
                out_cls.write(out)
            
            else:
                out = 'Visual inspection needed: Most likely simultaneous ascent.\n'
                out_cls.write(out)
        
        elif (dn1 < 0) and (dn2 < 0):
            
            if ((site[index_z[0]][1] == Lv0_1) or (site[index_z[0]][1] == Lv0_2)):
                out = 'Class = Vacancy insertion\nAtom type = ' + str(Type1) + '\n'
                out_cls.write(out)

            elif ((site[index_z[1]][1] == Lv0_1) or (site[index_z[1]][1] == Lv0_2)):
                out = 'Class = Vacancy insertion\nAtom type = ' + str(Type2) + '\n'
                out_cls.write(out)
            
            else:
                out = 'Visual inspection needed: Most likely simultaneous descent.\n'
                out_cls.write(out)
        
        else:
            
            if N_aux != 0:
                out = 'Class = Indirect exchange\nAtom type = ' + str(Type1) + '+' + str(Type2) + '\n'
                out_cls.write(out)
            
            else:    # Distance check
                
                for atom in range(0,N_dump):
                    if xyz1.finaldict[0][1]['id'][atom] == ID1:
                        index1 = atom
                    elif xyz1.finaldict[0][1]['id'][atom] == ID2:
                        index2 = atom
                
                x1 = xyz1.finaldict[step[tf]][1]['xu'][index1]
                y1 = xyz1.finaldict[step[tf]][1]['yu'][index1]
                z1 = xyz1.finaldict[step[tf]][1]['z'][index1]
                r1 = np.array([x1, y1, z1])
                
                x2 = xyz1.finaldict[step[tf]][1]['xu'][index2]
                y2 = xyz1.finaldict[step[tf]][1]['yu'][index2]
                z2 = xyz1.finaldict[step[tf]][1]['z'][index2]
                r2 = np.array([x2, y2, z2])
                
                dr = np.subtract(r1,r2)
                
                if np.linalg.norm(dr) > 1.10*dr_avg:
                    out = 'Class = Indirect exchange\nAtom type = ' + str(Type1) + '+' + str(Type2) + '\n' + 'Visual inspection advised: Missing auxiliary atom.\n'
                    out_cls.write(out)
                else:
                    out = 'Class = Direct exchange\nAtom type = ' + str(Type1) + '+' + str(Type2) + '\n'
                    out_cls.write(out)
    
    # More than 2 z-active atoms = Interlayer exchange vs. indeterminate
    elif N_z > 2:
        
        N_sub = 0
        for index, i in enumerate(index_z):
            
            if Lv_sub in site[i]:
                N_sub += 1
                index_last = index
            
            if index == len(index_z)-1:
                
                if N_sub == 0:
                    out = 'Visual inspection needed: More than two z-active atoms. Most likely conjoined events, or extra fluctuating atoms.\n'
                    out_cls.write(out)
                
                elif N_sub == 1:                    
                    Type = ev_atom[index_last][1]
                    out = 'Class = Interlayer transfer\nAtom type = ' + str(Type) + '\n'
                    out_cls.write(out)

                else:
                    out = 'Class = Interlayer transfer\nAtom type = Multi\n'
                    out_cls.write(out)

    out_cls.write('\n')
    
out_cls.close()
ls.close()

print('Event classification done > event_class.txt\n')

## Event statistics

In [ ]:
file_cls = pwd + '/event_class.txt'    # List of event classes
cls = open(file_cls,'r')    # Load list of event classes
cls_lines = cls.readlines()

file_ind = pwd + '/event_indeterminate.txt'    # Indeterminate events
out_ind = open(file_ind,'w')

file_stat = pwd + '/event_stat.txt'    # Event statistics
out_stat = open(file_stat,'w')

N_event = 0    # Number of events
N_indeterminate = 0    # Number of indeterminate events
for line_index, line in enumerate(cls_lines):
    if 'Event' in line:
        N_event += 1
    elif 'Visual inspection needed:' in line:
        N_indeterminate += 1
        
        end = line_index
        for i in reversed(range(end)):
            if 'Event' in cls_lines[i]:
                start = i
                break
        
        for i in range(start,end+1):
            out_ind.write(cls_lines[i])
        out_ind.write('\n')
        
indeterminacy = 100*N_indeterminate/N_event
header1 = 'Indeterminacy = %.2f' % (indeterminacy) + '% (' + str(N_indeterminate) + ' events)\n'
out_stat.write(header1)

Class = ['Popout', 'Vacancy insertion', 'Exchange descent', 'Exchange ascent', 'Hopping descent', 'Hopping ascent', 'Indirect exchange', 'Direct exchange', 'Interlayer transfer']
if N_type == 1:
    hist = np.zeros((9,), dtype=int)
else:
    hist = np.zeros((21,), dtype=int)

for line_index, line in enumerate(cls_lines):    # Count number of events in each class
    if 'Class' in line:
        
        for C_index, C in enumerate(Class):
            if C in line:
                
                for Type_index, Type in enumerate(range(1,N_type+1)):
                    
                    if C_index <= 5:
                        if str(Type) in cls_lines[line_index+1]:
                            hist[2*C_index + Type_index] += 1
                            
                    elif 6 <= C_index <= 7:
                        
                        Atom_type = str(Type) + '+' + str(Type)
                        if Atom_type in cls_lines[line_index+1]:
                            hist[2*C_index + Type_index + (C_index-6)] += 1
                        
                        if Type_index == 1:
                            Atom_type1 = str(Type-1) + '+' + str(Type)
                            Atom_type2 = str(Type) + '+' + str(Type-1)
                            if (Atom_type1 in cls_lines[line_index+1]) or (Atom_type2 in cls_lines[line_index+1]):
                                hist[2*C_index + Type_index + (C_index-6) + 1] += 1
                    
                    else:
                        
                        if str(Type) in cls_lines[line_index+1]:
                            hist[2*C_index + Type_index + 2] += 1
                        
                        if Type_index == 1:
                            if 'Multi' in cls_lines[line_index+1]:
                                hist[-1] += 1
                    
total = np.sum(hist)
header2 = str(total) + ' out of ' + str(N_event) + ' events classified.\n\n'
out_stat.write(header2)

header3 = ['Event class', 'Atom type', '# of occurrence', '% frequency']
types = ['%%%is', '%%%is', '%%%ii', '%%%i.2f']
colwidth = max(max(len(h) for h in header3), max(len(c) for c in Class))
out_stat.write((('\t'.join(["%%%is" % colwidth] * len(header3))) % tuple(header3)) + "\n")

for C_index, C in enumerate(Class):
    for Type_index, Type in enumerate(range(1,N_type+1)):
        
        if C_index <= 5:
            number = hist[2*C_index + Type_index]
            freq = float(100*number/total)
            out = ('\t'.join(t % colwidth for t in types)) % (C, str(Type), number, freq)
            out += '\n'
            out_stat.write(out)
            
        elif 6 <= C_index <= 7:
            
            Atom_type = str(Type) + '+' + str(Type)
            number = hist[2*C_index + Type_index + (C_index-6)]
            freq = float(100*number/total)
            out = ('\t'.join(t % colwidth for t in types)) % (C, Atom_type, number, freq)
            out += '\n'
            out_stat.write(out)

            if Type_index == 1:
                Atom_type = str(Type-1) + '+' + str(Type)
                number = hist[2*C_index + Type_index + (C_index-6) + 1]
                freq = float(100*number/total)
                out = ('\t'.join(t % colwidth for t in types)) % (C, Atom_type, number, freq)
                out += '\n'
                out_stat.write(out)
        
        else:
            
            number = hist[2*C_index + Type_index + 2]
            freq = float(100*number/total)
            out = ('\t'.join(t % colwidth for t in types)) % (C, str(Type), number, freq)
            out += '\n'
            out_stat.write(out)

            if Type_index == 1:
                Atom_type = 'Multi'
                number = hist[-1]
                freq = float(100*number/total)
                out = ('\t'.join(t % colwidth for t in types)) % (C, Atom_type, number, freq)
                out += '\n'
                out_stat.write(out)

out_stat.close()
out_ind.close()
cls.close()

print('Event statistics done > event_stat.txt\n')

## LAMMPS NEB preparation

In [ ]:
timestep = float(input('Enter the timestep used for the simulation, in ps : '))
dmp_step = 0.1    # Dump frequency in ps
d_step = int(dmp_step/timestep)    # Number of steps in a 0.1 ps interval

# Header for LAMMPS DUMP file of initial trajectory
l_step = 'ITEM: TIMESTEP'
l_atom = 'ITEM: NUMBER OF ATOMS'
l_box = 'ITEM: BOX BOUNDS xy xz yz pp pp ff'
l_xyz = 'ITEM: ATOMS id type x y z'

print('Preparing LAMMPS NEB images in ./neb/ev_***/ for each event...')

ls = open(file_ls,'r')    # Load list of events
ls_lines = ls.readlines()
ls_lines = [line.split() for line in ls_lines]

log = open(file_eq,'r')
log_lines = log.readlines()

for line_index, line in enumerate(ls_lines):
    if line:
        if (line_index > 9) and (len(line) == 3):
        
            ev_no = line[0]
            tf = line[1]
            dt = line[2]

            tf_step = int(tf/timestep)    # Number of 0.1 ps steps
            ti_step = tf_step - int(dt/timestep)

            ev_dir = pwd + '/ev_' + str(ev_no)    # Create NEB directory for each event
            os.mkdir(ev_dir)

            file_dmp = ev_dir + '/initial.dmp'    # LAMMPS DUMP file for visualizing initial trajectory
            out_dmp = open(file_dmp,'w')

            step = range(ti_step, tf_step+1, d_step)    # Event interval
            for s_index, s in enumerate(step):

                if s_index == 0:

                    file_img0 = ev_dir + '/img0.dat'    # LAMMPS DATA file for initial structure (image 0)
                    out_img0 = open(file_img0,'w')

                    for l_index, l in enumerate(log_lines):

                        if l_index <= head:
                            out_img0.write(l)

                    for atom in range(0,N_dump):

                        ID = xyz2.finaldict[s][1]['id'][atom]
                        Type = xyz2.finaldict[s][1]['type'][atom]
                        x = xyz2.finaldict[s][1]['x'][atom]
                        y = xyz2.finaldict[s][1]['y'][atom]
                        z = xyz2.finaldict[s][1]['z'][atom]

                        out = '%i\t%i\t%f\t%f\t%f\n'%(ID, Type, x, y, z)
                        out_img0.write(out)

                    out_img0.close()

                header = '%s\n%i\n%s\n%i\n%s\n%f %f %f\n%f %f %f\n%f %f %f\n%s\n'%(l_step, s, l_atom, N_dump, l_box, xlo_ext, xhi_ext, xy, ylo, yhi, xz, zlo, zhi, yz, l_xyz)
                out_dmp.write(header)

                file_trj = ev_dir + '/coords.initial.' + str(s_index)    # Images 1 to N
                out_trj = open(file_trj,'w')

                header = '# ID \t x \t y \t z \n'
                N_line = '%i\n'%(N_dump)
                out_trj.write(header)
                out_trj.write(N_line)

                for atom in range(0,N_dump):

                    ID = xyz2.finaldict[s][1]['id'][atom]
                    Type = xyz2.finaldict[s][1]['type'][atom]
                    x = xyz2.finaldict[s][1]['x'][atom]
                    y = xyz2.finaldict[s][1]['y'][atom]
                    z = xyz2.finaldict[s][1]['z'][atom]

                    out = '%i %i %f %f %f\n'%(ID, Type, x, y, z)
                    out_dmp.write(out)

                    out = '%i\t%f\t%f\t%f\n'%(ID, x, y, z)
                    out_trj.write(out)

                out_trj.close()

            out_dmp.close()

ls.close()
log.close()

print('LAMMPS NEB images generated > ./ev_***/')
print('Use initial.dmp to visualize the initial trajectory.')
print('Use img0.dat as the initial structure.')
print('Use coords.initial.** as the NEB images.\n')

print('Once the NEB calculations converge, NEB2DFT.ipynb or NEB2ASE.ipynb can be used to prepare DFT or ASE optimization, respectively.')